In [ ]:
"""
Visualization of non-benign connections data
Developed with Python version 3.10.7
"""

In [ ]:
# Load the top modules that are used in multiple places
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, precision_recall_fscore_support

%matplotlib inline  

In [ ]:
# Column name mapping from original data to compact form
# All the X** are features
feature_map = {
 ' Destination Port' : 'X1',
 ' Flow Duration' : 'X2', 
 ' Total Fwd Packets' : 'X3', 
 ' Total Backward Packets' : 'X4', 
 'Total Length of Fwd Packets' : 'X5', 
 ' Total Length of Bwd Packets' : 'X6', 
 ' Fwd Packet Length Max' : 'X7', 
 ' Fwd Packet Length Min' : 'X8', 
 ' Fwd Packet Length Mean' : 'X9', 
 ' Fwd Packet Length Std' : 'X10', 
 'Bwd Packet Length Max' : 'X11', 
 ' Bwd Packet Length Min' : 'X12', 
 ' Bwd Packet Length Mean' : 'X13', 
 ' Bwd Packet Length Std' : 'X14', 
 'Flow Bytes/s' : 'X15', 
 ' Flow Packets/s' : 'X16', 
 ' Flow IAT Mean' : 'X17', 
 ' Flow IAT Std' : 'X18', 
 ' Flow IAT Max' : 'X19', 
 ' Flow IAT Min' : 'X20', 
 'Fwd IAT Total' : 'X21', 
 ' Fwd IAT Mean' : 'X22', 
 ' Fwd IAT Std' : 'X23', 
 ' Fwd IAT Max' : 'X24', 
 ' Fwd IAT Min' : 'X25', 
 'Bwd IAT Total' : 'X26', 
 ' Bwd IAT Mean' : 'X27', 
 ' Bwd IAT Std' : 'X28', 
 ' Bwd IAT Max' : 'X29', 
 ' Bwd IAT Min' : 'X30', 
 'Fwd PSH Flags' : 'X31', 
 ' Bwd PSH Flags' : 'X32', 
 ' Fwd URG Flags' : 'X33', 
 ' Bwd URG Flags' : 'X34', 
 ' Fwd Header Length' : 'X35', 
 ' Bwd Header Length' : 'X36', 
 'Fwd Packets/s' : 'X37', 
 ' Bwd Packets/s' : 'X38', 
 ' Min Packet Length' : 'X39', 
 ' Max Packet Length' : 'X40', 
 ' Packet Length Mean' : 'X41', 
 ' Packet Length Std' : 'X42', 
 ' Packet Length Variance' : 'X43', 
 'FIN Flag Count' : 'X44', 
 ' SYN Flag Count' : 'X45', 
 ' RST Flag Count' : 'X46', 
 ' PSH Flag Count' : 'X47', 
 ' ACK Flag Count' : 'X48', 
 ' URG Flag Count' : 'X49', 
 ' CWE Flag Count' : 'X50', 
 ' ECE Flag Count' : 'X51', 
 ' Down/Up Ratio' : 'X52', 
 ' Average Packet Size' : 'X53', 
 ' Avg Fwd Segment Size' : 'X54', 
 ' Avg Bwd Segment Size' : 'X55', 
 ' Fwd Header Length.1' : 'X56', 
 'Fwd Avg Bytes/Bulk' : 'X57', 
 ' Fwd Avg Packets/Bulk' : 'X58', 
 ' Fwd Avg Bulk Rate' : 'X59', 
 ' Bwd Avg Bytes/Bulk' : 'X60', 
 ' Bwd Avg Packets/Bulk' : 'X61', 
 'Bwd Avg Bulk Rate' : 'X62', 
 'Subflow Fwd Packets' : 'X63', 
 ' Subflow Fwd Bytes' : 'X64', 
 ' Subflow Bwd Packets' : 'X65', 
 ' Subflow Bwd Bytes' : 'X66', 
 'Init_Win_bytes_forward' : 'X67', 
 ' Init_Win_bytes_backward' : 'X68', 
 ' act_data_pkt_fwd' : 'X69', 
 ' min_seg_size_forward' : 'X70', 
 'Active Mean' : 'X71', 
 ' Active Std' : 'X72', 
 ' Active Max' : 'X73', 
 ' Active Min' : 'X74', 
 'Idle Mean' : 'X75', 
 ' Idle Std' : 'X76', 
 ' Idle Max' : 'X77', 
 ' Idle Min' : 'X78', 
}


feature_map2 = {} 
for i in feature_map:
    feature_map2.setdefault(feature_map[i], []).append(i)

num_ids_features = len(feature_map)
feature_names = list(feature_map.values())

In [ ]:
# label names (YY) in the data and their
# mapping to numerical values
label_map = {
 'BENIGN' : 1,
 'FTP-Patator' : 2,
 'SSH-Patator' : 3,
 'DoS slowloris' : 4,
 'DoS Slowhttptest': 5,
 'DoS Hulk' : 6,
 'DoS GoldenEye' : 7,
 'Heartbleed' : 8,
 'Web Attack � Brute Force' : 9,
 'Web Attack � XSS' : 10,
 'Web Attack � Sql Injection' : 11,
 'Infiltration' : 12,
 'Bot' : 13,
 'PortScan' : 14,
 'DDoS' : 15,
}

ids_classes = list(label_map.keys())
num_ids_classes = len(label_map)
num_mal_classes = num_ids_classes - 1
mal_class_num = np.arange(1, num_mal_classes + 1)

In [ ]:
outdir = './MachineLearningCVE/restart/'
mal_data = 'mal.csv'

In [ ]:
df = pd.read_csv(outdir + mal_data)

In [ ]:
df.head()

In [ ]:
"""
Some rows with 'YY' = 5 have the 'X15' column as NaN.
Set these to the average value of the 'X15' column for 'YY' = 5
"""
# Columns containing nan
df.columns[df.isna().any()].tolist()

# Only 'YY' = 5 has NaNs
(df[df['X15'].isna()])['YY'].unique()

avg_x15 = df[df['YY'] == 5]['X15'].mean()
df.fillna(avg_x15, inplace=True)

if len(df[df['X15'].isna()]) == 0:
    print ('NaNs replaced with mean')
else:
    print ('something went wrong with NaN handling')

In [ ]:
"""
Some rows with 'YY' = 12, 13, 14, 1, 5 have inf in the X15 and X16 columns
Replace these with the column max
"""
# Columns containing inf
df.columns.to_series()[np.isinf(df).any()]

# labels corresponding to rows with inf
df.iloc[df.index[np.isinf(df).any(1)]]['YY'].unique()

# replace infs in x15 column with max non-inf value
max_x15 = df.loc[df['X15'] != np.inf, 'X15'].max()
print (max_x15)
df['X15'].replace(np.inf, max_x15, inplace=True)

# replace infs in x16 column with max non-inf value
max_x16 = df.loc[df['X16'] != np.inf, 'X16'].max()
print (max_x16)
df['X16'].replace(np.inf, max_x16, inplace=True)

if len(df.columns.to_series()[np.isinf(df).any()]) == 0:
    print ('infs replaced with max')
else:
    print ('something went wrong with inf handling')

In [ ]:
"""
The dataset is badly skewed.
"""
df.groupby(['YY'])['YY'].count()

In [ ]:
"""
Get the medians of the 14 classes across all dimensions.


df_median = df.groupby(['YY'])[feature_names].median()

print (df_median.shape)
print (df_median)
"""

In [ ]:
"""
PCA
Plot them in 2D.

from sklearn.decomposition import PCA

mod_pca = PCA(n_components=10)
X_pca = mod_pca.fit_transform(df_median)
print (X_pca.shape)

fig, ax = plt.subplots()
ax.scatter(X_pca[:, 4], X_pca[:, 5])

for i, txt in enumerate(mal_class_num):
    ax.annotate(txt, (X_pca[i, 4], X_pca[i, 5]))
    
"""

In [ ]:
"""
Multi Dimensional Scaling (MDS)
Plot them in 2D.

from sklearn.manifold import MDS

mod_mds = MDS(n_components=3, n_init=10, verbose=1, random_state=42)
X_mds = mod_mds.fit_transform(df_median)
print (X_mds.shape)

fig, ax = plt.subplots()
ax.scatter(X_mds[:, 0], X_mds[:, 1])

for i, txt in enumerate(mal_class_num):
    ax.annotate(txt, (X_mds[i, 0], X_mds[i, 1]))
"""

In [ ]:
"""
Logistic regression to understand the most important parameters
"""
n_iter = 100
imp_cutoff = 3.75 # based on trial error to reduce the total number of features studied

In [ ]:
# Logistic regression with all features

mod_scaler = StandardScaler()
X_std = mod_scaler.fit_transform(df[feature_names])
mod_log = LogisticRegression(max_iter=n_iter)
mod_log.fit(X_std, df['YY'])

y_pred = mod_log.predict(X_std)


In [ ]:
importances = pd.DataFrame(data={
    'Attribute': feature_map.keys(),
    'Importance': mod_log.coef_[0]
})
# importances = importances.sort_values(by='Importance', ascending=False)

plt.rcParams["figure.figsize"] = (6,12)
plt.barh(y=importances['Attribute'], width=importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=12)
plt.xticks(rotation='vertical')
# save figure to file
plt.savefig('Fig1.eps', format='eps', dpi=1200, bbox_inches="tight")
plt.show()

In [ ]:
# the six most important features
idxs = np.argwhere(abs(mod_log.coef_[0]) > imp_cutoff).flatten().tolist()
print (idxs)
imp_feats = ['X' + str(i+1) for i in idxs]
print (imp_feats)
imp_names = [feature_map2[k][0] for k in imp_feats]
print (imp_names)

In [ ]:
# Tested with python 3.10.7
plt.rcParams["figure.figsize"] = (20,10)

cm = confusion_matrix(df['YY'], y_pred, labels=mod_log.classes_, normalize='true')
cm = cm.round(decimals=2)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=mod_log.classes_)
disp.plot()


In [ ]:
# f1 score only
# print (f1_score(df['YY'], y_pred, average='weighted'))

# precision recall f1
precision_recall_fscore_support(df['YY'], y_pred, average='weighted')

In [ ]:
# rererun the logistic regression with the same number of iterations but only with the top features
mod_scaler_t = StandardScaler()
X_std_t = mod_scaler_t.fit_transform(df[imp_feats])
mod_log_t = LogisticRegression(max_iter=n_iter)
mod_log_t.fit(X_std_t, df['YY'])

y_pred_t = mod_log_t.predict(X_std_t)


In [ ]:
# f1 score only
# print (f1_score(df['YY'], y_pred_t, average='weighted'))

# precision recall f1
precision_recall_fscore_support(df['YY'], y_pred_t, average='weighted')

In [ ]:
# box plots for important features
plt.rcParams["figure.figsize"] = (8,8)
# index for saving figures to file; starting at 2
j = 2
for f in imp_feats:
    bl = []
    fig, ax = plt.subplots()
    for m in range(1, num_ids_classes+1):
        bl.append(df[df['YY'] == m][f])
    ax.set_title(feature_map2[f][0])
    ax.boxplot(bl, vert=False, showfliers=False)
    fig_name = 'Fig' + str(j) + '.eps'
    plt.savefig(fig_name, format='eps', dpi=1200, bbox_inches="tight")
    plt.show()
    j += 1